In [1]:
import sympy as sym
from IPython.display import display, Math, Latex
import pickle

In [2]:
r,theta,z = sym.symbols('r,theta,z')

In [3]:
#sigma_rr,sigma_tt,sigma_zz,E,nu,epsilon_rr,epsilon_tt,epsilon_zz = sym.Funtion('sigma_rr,sigma_tt,sigma_zz,E,nu,epsilon_rr,epsilon_tt,epsilon_zz')(r)
sigma_rr = sym.Function('sigma_rr')(r)
sigma_tt = sym.Function('sigma_tt')(r)
sigma_zz = sym.Function('sigma_zz')(r)
epsilon_rr = sym.Function('epsilon_rr')(r)
epsilon_tt = sym.Function('epsilon_tt')(r)
epsilon_zz = sym.Function('epsilon_zz')(r)
E,nu = sym.symbols('E,nu')




In [4]:
sigma_zz = 0 #plane stress condition
eq1 = sym.Eq(epsilon_rr,1/E*(sigma_rr - nu*(sigma_tt)))
eq2 = sym.Eq(epsilon_tt,1/E*(sigma_tt - nu*(sigma_rr)))
eq3 = sym.Eq(epsilon_zz,1/E*(-nu*(sigma_rr + sigma_tt)))

display(eq1,eq2,eq3)

Eq(epsilon_rr(r), (-nu*sigma_tt(r) + sigma_rr(r))/E)

Eq(epsilon_tt(r), (-nu*sigma_rr(r) + sigma_tt(r))/E)

Eq(epsilon_zz(r), -nu*(sigma_rr(r) + sigma_tt(r))/E)

In [5]:
soln = sym.solve([eq1,eq2],[sigma_rr,sigma_tt])
soln = sym.factor(soln)
display(soln)

{sigma_rr(r): -E*(nu*epsilon_tt(r) + epsilon_rr(r))/((nu - 1)*(nu + 1)), sigma_tt(r): -E*(nu*epsilon_rr(r) + epsilon_tt(r))/((nu - 1)*(nu + 1))}

In [6]:
u = sym.Function('u')(r)
sym.diff(u,r)

Derivative(u(r), r)

In [7]:

soln2 = soln.subs({epsilon_rr: sym.diff(u,r), epsilon_tt: u/r})
sigma_rr = soln2[sigma_rr]
sigma_tt = soln2[sigma_tt]
display(sigma_rr)
display(sigma_tt)


-E*(nu*u(r)/r + Derivative(u(r), r))/((nu - 1)*(nu + 1))

-E*(nu*Derivative(u(r), r) + u(r)/r)/((nu - 1)*(nu + 1))

In [8]:
cond = (sym.diff(sigma_rr,r) + (sigma_rr - sigma_tt)/r).simplify()
eq6 = sym.Eq(cond,0)
display(eq6)


Eq(E*(-r**2*Derivative(u(r), (r, 2)) - r*Derivative(u(r), r) + u(r))/(r**2*(nu**2 - 1)), 0)

In [9]:
soln3 = sym.dsolve(eq6,u)
display(soln3)

Eq(u(r), C1/r + C2*r)

In [10]:
C1,C2 = sym.symbols('C1,C2')
u_soln = C1/r + C2*r
sigma_rr = sym.factor(sigma_rr.replace(u,u_soln).doit().simplify())
sigma_tt = sym.factor(sigma_tt.replace(u,u_soln).doit().simplify())
display(sigma_rr,sigma_tt)





-E*(C1*nu - C1 + C2*nu*r**2 + C2*r**2)/(r**2*(nu - 1)*(nu + 1))

-E*(-C1*nu + C1 + C2*nu*r**2 + C2*r**2)/(r**2*(nu - 1)*(nu + 1))

In [11]:
r_o,r_i,p_o,p_i = sym.symbols('r_o,r_i,p_o,p_i')

In [12]:
bc1 = sym.Eq(sigma_rr.subs(r,r_i),-p_i)
bc2= sym.Eq(sigma_rr.subs(r,r_o),-p_o)
display(bc1,bc2)

Eq(-E*(C1*nu - C1 + C2*nu*r_i**2 + C2*r_i**2)/(r_i**2*(nu - 1)*(nu + 1)), -p_i)

Eq(-E*(C1*nu - C1 + C2*nu*r_o**2 + C2*r_o**2)/(r_o**2*(nu - 1)*(nu + 1)), -p_o)

In [13]:
soln4 = sym.factor(sym.solve([bc1,bc2],[C1,C2]))
sigma_rr_soln = sym.factor(sigma_rr.subs([(C1,soln4[C1]),(C2,soln4[C2])]).simplify()).expand()
sigma_tt_soln = sym.factor(sigma_tt.subs([(C1,soln4[C1]),(C2,soln4[C2])]).simplify()).expand()
sigma_rr_soln = sym.collect(sigma_rr_soln,r)
sigma_tt_soln = sym.collect(sigma_tt_soln,r)
display(sigma_rr_soln,sigma_tt_soln)




-p_i*r_i**2/(r_i**2 - r_o**2) + p_o*r_o**2/(r_i**2 - r_o**2) + (p_i*r_i**2*r_o**2/(r_i**2 - r_o**2) - p_o*r_i**2*r_o**2/(r_i**2 - r_o**2))/r**2

-p_i*r_i**2/(r_i**2 - r_o**2) + p_o*r_o**2/(r_i**2 - r_o**2) + (-p_i*r_i**2*r_o**2/(r_i**2 - r_o**2) + p_o*r_i**2*r_o**2/(r_i**2 - r_o**2))/r**2

In [14]:
I1 = sym.factor((sigma_rr_soln + sigma_tt_soln).simplify())
display(I1)

-2*(p_i*r_i**2 - p_o*r_o**2)/((r_i - r_o)*(r_i + r_o))

In [15]:
display(repr(sigma_rr_soln),repr(sigma_tt_soln))
    

'-p_i*r_i**2/(r_i**2 - r_o**2) + p_o*r_o**2/(r_i**2 - r_o**2) + (p_i*r_i**2*r_o**2/(r_i**2 - r_o**2) - p_o*r_i**2*r_o**2/(r_i**2 - r_o**2))/r**2'

'-p_i*r_i**2/(r_i**2 - r_o**2) + p_o*r_o**2/(r_i**2 - r_o**2) + (-p_i*r_i**2*r_o**2/(r_i**2 - r_o**2) + p_o*r_i**2*r_o**2/(r_i**2 - r_o**2))/r**2'